# Bu skriptimdə 'fraud_data' adlı datada boosting methods modelləri(XGBoost, LighGBM, CatBoost) və RandomForest modelini istifadə etmişəm. 

In [1]:
#İlk öncə modelləri tətbiq etmək üçün ilkin proseslərdə istifadə olunacaq kitabxanaları daxil etmişəm. Bu kitabxanalar 'import' vasitəsilə daxil olunur.
#'as' vasitəsilə kitabxaların adını qisaldılmış formada yazmışam ki, daha sonra istifadə etdikdə rahat olsun.
#sns.set() hər hansı qrafik yaratdıqda arxa fonun digərlərindən fərqli olub görünməsi üçün istifadə olunur

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
#datanı oxuduruq

data=pd.read_excel(r'fraud_data.xlsx')
data

,type,branch,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,unusuallogin,isFlaggedFraud,Acct type,Date of transaction,Time of day,isFraud
0,PAYMENT,Indonesia,9839.64,C1231006815,170136.0,160296.36,M1979787155,9,0,Current,3/1/2018,Morning,0
1,PAYMENT,India,1864.28,C1666544295,21249.0,19384.72,M2044282225,10,0,Savings,5/1/2018,Morning,0
2,TRANSFER,India,181.00,C1305486145,181.0,0.00,C553264065,2,0,Current,7/1/2018,Morning,1
3,CASH_OUT,Australia,181.00,C840083671,181.0,0.00,C38997010,1,0,Current,6/1/2018,Afternoon,1
4,PAYMENT,Australia,11668.14,C2048537720,41554.0,29885.86,M1230701703,17,0,Current,6/1/2018,Morning,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10120,PAYMENT,Cuba,337.50,C1494306005,33107.0,32769.50,M1424027000,7,0,Current,3/1/2018,Afternoon,0
10121,PAYMENT,Mexico,5003.57,C1633890169,32769.5,27765.93,M1854745805,11,0,Savings,5/1/2018,Morning,0
10122,PAYMENT,Panama,10424.89,C1026138669,50780.0,40355.11,M1852900317,6,0,Savings,7/1/2018,Night,0
10123,PAYMENT,Mexico,2823.59,C378659213,986.0,0.00,M301812950,12,0,Savings,6/1/2018,Night,0


In [3]:
#Deskriptiv statistikaya baxdıqda  data haqqında müəyyən fikir formalaşır.
data.describe(include='all')

,type,branch,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,unusuallogin,isFlaggedFraud,Acct type,Date of transaction,Time of day,isFraud
count,10121,10125,1.012300e+04,10119,1.012400e+04,1.012500e+04,10119,10125.000000,10125.0,10115,10118,10123,10125.000000
unique,5,135,NaN,10119,NaN,NaN,6494,NaN,NaN,2,14,3,NaN
top,PAYMENT,Estados Unidos,NaN,C1231006815,NaN,NaN,C985934102,NaN,NaN,Savings,6/1/2018,Afternoon,NaN
freq,5540,1282,NaN,1,NaN,NaN,62,NaN,NaN,6985,1446,3626,NaN
mean,NaN,NaN,1.049029e+05,NaN,8.837822e+05,9.046314e+05,NaN,10.513580,0.0,NaN,NaN,NaN,0.011654
std,NaN,NaN,2.706610e+05,NaN,2.124641e+06,2.170130e+06,NaN,5.809393,0.0,NaN,NaN,NaN,0.107330
min,NaN,NaN,2.390000e+00,NaN,0.000000e+00,0.000000e+00,NaN,0.000000,0.0,NaN,NaN,NaN,0.000000
25%,NaN,NaN,4.397480e+03,NaN,1.290000e+02,0.000000e+00,NaN,6.000000,0.0,NaN,NaN,NaN,0.000000
50%,NaN,NaN,1.279831e+04,NaN,2.136337e+04,1.019179e+04,NaN,10.000000,0.0,NaN,NaN,NaN,0.000000
75%,NaN,NaN,1.143832e+05,NaN,1.724414e+05,1.707442e+05,NaN,16.000000,0.0,NaN,NaN,NaN,0.000000


In [4]:
#Bu datada 'Date of transaction' sütünu date formatında olmadığı üçün formatını dəyişib il, ay, gün sütunlarını yaratmışam.

data['Date of transaction']=pd.to_datetime(data['Date of transaction'], dayfirst=True)

In [5]:
data['transaction_year']=data['Date of transaction'].dt.year
data['transaction_month']=data['Date of transaction'].dt.month
data['transaction_day']=data['Date of transaction'].dt.day

In [6]:
#Sütunun özündən 3 dənə sütün yaratdığım üçün həmin sütunu silirəm.

data.drop(columns='Date of transaction', axis=1, inplace=True)

In [7]:
#Deskriptiv statistikaya baxdıqda bu 3 sütunun unique dəyərləri çox olduğu üçün modeldə istifadə etmək düzgün deyil. Bunun üçün silirəm.

data=data.drop(columns=data[['nameOrig', 'nameDest', 'branch']], axis=1)

In [169]:
data

,type,amount,oldbalanceOrg,newbalanceOrig,unusuallogin,isFlaggedFraud,Acct type,Time of day,isFraud,transaction_year,transaction_month,transaction_day
0,PAYMENT,9839.64,170136.0,160296.36,9,0,Current,Morning,0,2018.0,1.0,3.0
1,PAYMENT,1864.28,21249.0,19384.72,10,0,Savings,Morning,0,2018.0,1.0,5.0
2,TRANSFER,181.00,181.0,0.00,2,0,Current,Morning,1,2018.0,1.0,7.0
3,CASH_OUT,181.00,181.0,0.00,1,0,Current,Afternoon,1,2018.0,1.0,6.0
4,PAYMENT,11668.14,41554.0,29885.86,17,0,Current,Morning,0,2018.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10120,PAYMENT,337.50,33107.0,32769.50,7,0,Current,Afternoon,0,2018.0,1.0,3.0
10121,PAYMENT,5003.57,32769.5,27765.93,11,0,Savings,Morning,0,2018.0,1.0,5.0
10122,PAYMENT,10424.89,50780.0,40355.11,6,0,Savings,Night,0,2018.0,1.0,7.0
10123,PAYMENT,2823.59,986.0,0.00,12,0,Savings,Night,0,2018.0,1.0,6.0


In [8]:
#Datada olan missing value-ları yoxlayıram

data.isnull().sum()

type                  4
amount                2
oldbalanceOrg         1
newbalanceOrig        0
unusuallogin          0
isFlaggedFraud        0
Acct type            10
Time of day           2
isFraud               0
transaction_year      7
transaction_month     7
transaction_day       7
dtype: int64

In [9]:
#Get dummies metodu ilə datada kategorik sütunların numerik sütünlara çevirərək başqa dataya mənimsədirəm.
#Çünki istifadə edəcəyim modellər yalnız numerik dəyərləri qəbul edir.

data_dummy=pd.get_dummies(data, drop_first=True)

In [10]:
data_dummy

,amount,oldbalanceOrg,newbalanceOrig,unusuallogin,isFlaggedFraud,isFraud,transaction_year,transaction_month,transaction_day,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,Acct type_Savings,Time of day_Morning,Time of day_Night
0,9839.64,170136.0,160296.36,9,0,0,2018.0,1.0,3.0,False,False,True,False,False,True,False
1,1864.28,21249.0,19384.72,10,0,0,2018.0,1.0,5.0,False,False,True,False,True,True,False
2,181.00,181.0,0.00,2,0,1,2018.0,1.0,7.0,False,False,False,True,False,True,False
3,181.00,181.0,0.00,1,0,1,2018.0,1.0,6.0,True,False,False,False,False,False,False
4,11668.14,41554.0,29885.86,17,0,0,2018.0,1.0,6.0,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10120,337.50,33107.0,32769.50,7,0,0,2018.0,1.0,3.0,False,False,True,False,False,False,False
10121,5003.57,32769.5,27765.93,11,0,0,2018.0,1.0,5.0,False,False,True,False,True,True,False
10122,10424.89,50780.0,40355.11,6,0,0,2018.0,1.0,7.0,False,False,True,False,True,False,True
10123,2823.59,986.0,0.00,12,0,0,2018.0,1.0,6.0,False,False,True,False,True,False,True


In [11]:
data_dummy.columns

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'unusuallogin',
       'isFlaggedFraud', 'isFraud', 'transaction_year', 'transaction_month',
       'transaction_day', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT',
       'type_TRANSFER', 'Acct type_Savings', 'Time of day_Morning',
       'Time of day_Night'],
      dtype='object')

In [12]:
data.columns

Index(['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'unusuallogin',
       'isFlaggedFraud', 'Acct type', 'Time of day', 'isFraud',
       'transaction_year', 'transaction_month', 'transaction_day'],
      dtype='object')

In [13]:
#Datanın asılı dəyişənini müəyyən edib target adlı dəyişənə mənimsədirəm. Digər sütünların hər birini modeldə müstəqil dəyişən 
#olaraq istifadə edəcəyim üçün inputs adlı dəyişənə mənimsədirəm

inputs=data_dummy.drop('isFraud', axis=1)
target=data_dummy['isFraud']

In [14]:
#datanı train və test hissəyə bölmək üçün train_test_split kitabxanasını daxil edirəm

from sklearn.model_selection import train_test_split

#x_train, x_test, y_train, y_test olaraq bölürəm
x_train, x_test, y_train, y_test=train_test_split(inputs, target, test_size=0.3, random_state=42)

In [15]:
data.dtypes

type                  object
amount               float64
oldbalanceOrg        float64
newbalanceOrig       float64
unusuallogin           int64
isFlaggedFraud         int64
Acct type             object
Time of day           object
isFraud                int64
transaction_year     float64
transaction_month    float64
transaction_day      float64
dtype: object

In [16]:
for i in data.columns:
    if data[i].dtypes==object:
        print(i)

type
Acct type
Time of day


In [22]:
#CatBoost modeli numerik dəyərlərlə yanaşı kategorik sütünlarıda qəbul etdiyi üçün kategorik formaya uyğun olaraq əlavə input və target
#yaradıram

inputs_cat=data.drop('isFraud', axis=1)
target_cat=data['isFraud']

In [23]:
#CatBoost modelində kategoric sütünlardan istifadə edirəmsə həmin sütunlarda missing dəyərlər olmamalıdır

columns_to_fill=['type', 'Acct type', 'Time of day']
inputs_cat[columns_to_fill]=inputs_cat[columns_to_fill].fillna(value='Missing_value')

In [24]:
#kategorik sütunlar üçün train test split edirik
x_train_cat, x_test_cat, y_train_cat, y_test_cat=train_test_split(inputs_cat, target_cat, test_size=0.3, random_state=42)

In [25]:
x_train_cat.columns

Index(['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'unusuallogin',
       'isFlaggedFraud', 'Acct type', 'Time of day', 'transaction_year',
       'transaction_month', 'transaction_day'],
      dtype='object')

In [26]:
#modelleri import edirik
#import etmek üçün ilk öncə install etmek lazımdır
#pip install xgboost
#pip install lightgbm
#pip install catboost

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [32]:
#data klassifikasiya datası olduğu üçün accuracy olaraq gini score baxmaq lazımdır. bunun üçün roc_auc-score daxil edirik

from sklearn.metrics import roc_auc_score 

#bir neçə model tətbiq etdiyim üçün hər birini ayrı ayrılıqda hesablamayıb bir funksiya içərisində həm train edirəm, həm də gini dəyərini hesablayıram

def train_and_evaluate_model(model_name, model, x_train, y_train, x_test, y_test):

        model.fit(x_train, y_train)

        y_pred = model.predict(x_test)
        y_prob = model.predict_proba(x_test)[:, 1]

        roc_prob = roc_auc_score(y_test, y_prob)
        gini_prob = roc_prob * 2 - 1
        

        print(f'Model Performance for {model_name}')
        print('Gini prob is', gini_prob * 100)
        

        return gini_prob

In [34]:
#modelləri bir listə yığıb pipeline-dan çagırıram

models=[]

xgb_def_model=XGBClassifier()
lgb_def_model=LGBMClassifier()
ct_def_model=CatBoostClassifier()
ct_cat_def_model=CatBoostClassifier(cat_features=['type', 'Acct type', 'Time of day'])
rf_def=RandomForestClassifier()

models.extend([
    ('XGB_def',xgb_def_model),
    ('LGBM_def',lgb_def_model),
    ('CatBoost_def',ct_def_model),
    ('CatBoost_cat_def',ct_cat_def_model),
    ('RandomForest_def', rf_def), 
])

In [35]:
#hər bir modelin gini dəyərini görüb bir-biri ilə müqayisə etmək üçün bir dataframe-ə yığıram.

gini_df=pd.DataFrame(columns=['model', 'gini'])

for model_name, model in models:
    if model_name=='CatBoost_cat_def':
        gini=train_and_evaluate_model(model_name, model, x_train_cat, y_train_cat, x_test_cat, y_test_cat)
    elif model_name=='RandomForest_def':
        x_train_filled=x_train.copy()
        x_test_filled=x_test.copy()
        for i in x_train_filled.columns:
            if x_train_filled[i].dtypes==object:
                x_train_filled[i]=x_train_filled[i].fillna(value=x_train_filled[i].mode()[0])
                x_test_filled[i]=x_test_filled[i].fillna(value=x_test_filled[i].mode()[0])
            else:
                x_train_filled[i]=x_train_filled[i].fillna(value=x_train_filled[i].mean())
                x_test_filled[i]=x_test_filled[i].fillna(value=x_test_filled[i].mean())
        gini=train_and_evaluate_model(model_name, model, x_train_filled, y_train, x_test_filled, y_test)
    else:
        gini=train_and_evaluate_model(model_name, model, x_train, y_train, x_test, y_test)
    if gini is not None:
        gini_df=pd.concat([gini_df, pd.DataFrame({ 'model':[model_name], 'gini':[gini]})], ignore_index=True)
        
gini_df_sorted=gini_df.sort_values(by='gini', ascending=False)
gini_df_sorted

Model Performance for XGB_def
Gini prob is 62.40627217619548
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 79, number of negative: 7008
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000787 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 815
[LightGBM] [Info] Number of data points in the train set: 7087, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011147 -> initscore=-4.485360
[LightGBM] [Info] Start training from score -4.485360
Model Performance for LGBM_def
Gini prob is 66.61451252981763
Learning rate set to 0.023773
0:	learn: 0.6397361	total: 165ms	remaining: 2m 44s
1:	learn: 0.5889522	total: 168ms	remaining: 1m 23s
2:	learn: 0.5443831	total: 171ms	remaining: 56.7s
3:	learn: 0.5039216	total: 173ms	remaining: 43.1s
4:	learn: 0.4

,model,gini
1,LGBM_def,0.666145
4,RandomForest_def,0.663743
0,XGB_def,0.624063
2,CatBoost_def,0.614179
3,CatBoost_cat_def,0.575893


In [36]:
#hər bir modeli hyperparametr tuning etmək üçün optuna kitabxanasından istifadə edirəm
#ilk öncə install etmək lazımdır
#pip install optuna

import optuna

In [38]:
from sklearn.model_selection import cross_val_score

def best_params_for_model(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100)
    }

    lgb_clf = LGBMClassifier(**param)

    auc = cross_val_score(lgb_clf, x_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=10)

print('Best trial:')
lgb_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', lgb_params)

best_lgb_model = LGBMClassifier(**lgb_params)

[I 2024-05-04 22:02:19,088] A new study created in memory with name: no-name-2a4a2c14-5944-42e2-8c7b-8b941b80ffb6
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\1309682227.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
[I 2024-05-04 22:02:21,648] Trial 0 finished with value: 0.7303051701596223 and parameters: {'n_estimators': 532, 'learning_rate': 0.15564618391944715, 'max_depth': 9, 'num_leaves': 34}. Best is trial 0 with value: 0.7303051701596223.
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\1309682227.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_l

Best trial:
  Value: 0.742
  Params:  {'n_estimators': 119, 'learning_rate': 0.06023149379741604, 'max_depth': 4, 'num_leaves': 51}


In [40]:
def best_params_for_model(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5,1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5,1),
        'gamma': trial.suggest_int('gamma', 0,5)

}
    xgb_clf = XGBClassifier(**param)

    auc = cross_val_score(xgb_clf, x_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc



study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=10)

print('Best trial:')
xgb_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', xgb_params)

best_xgb_model = XGBClassifier(**xgb_params)

[I 2024-05-04 22:02:47,469] A new study created in memory with name: no-name-72e7a9db-24e8-4b2d-8d81-38f177876d09
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\1390477293.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\1390477293.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5,1),
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\1390477293.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Us

Best trial:
  Value: 0.797
  Params:  {'n_estimators': 790, 'learning_rate': 0.038646937267731225, 'max_depth': 7, 'subsample': 0.5251488658076695, 'colsample_bytree': 0.606508612126674, 'gamma': 5}


In [51]:
def best_params_for_model(trial):

    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10), 
        'loss_function': trial.suggest_categorical('loss_function', ['Logloss']) 
    }

    cb_clf = CatBoostClassifier(**param)

    auc = cross_val_score(cb_clf, x_train, y_train, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc



study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=10)

print('Best trial:')
ct_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', ct_params)

best_ct_model = CatBoostClassifier(**ct_params)

[I 2024-05-04 22:06:17,685] A new study created in memory with name: no-name-e449fd4b-7d3a-4d23-baeb-ab96b4c18739
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\889594177.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
C:\Users\admin\AppData\Local\Temp\ipykernel_13016\889594177.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10),
[I 2024-05-04 22:06:21,018] Trial 0 finished with value: 0.7302202041785376 and parameters: {'iterations': 409, 'learning_rate': 0.095089600789425, 'depth': 9, 'l2_leaf_reg': 0.26026209483035956, 'loss_fu

Best trial:
  Value: 0.781
  Params:  {'iterations': 387, 'learning_rate': 0.011927967381337634, 'depth': 7, 'l2_leaf_reg': 6.743790321315222, 'loss_function': 'Logloss'}


In [42]:
def best_params_for_model(trial):
    param = {
    'n_estimators' : trial.suggest_int('n_estimators', 100, 1000, step=100),
    'max_depth' : trial.suggest_int('max_depth', 3, 20),
    'min_samples_split' : trial.suggest_int('min_samples_split', 2, 20),
    'min_samples_leaf' : trial.suggest_int('min_samples_leaf', 1, 10),
    'max_features' : trial.suggest_categorical('max_features', [ 'sqrt', 'log2'])
    }

    rf_clf = RandomForestClassifier(**param)

    auc = cross_val_score(rf_clf, x_train_filled, y_train, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=1)

print('Best trial:')
rf_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', rf_params)

best_rf_model = RandomForestClassifier(**rf_params)

[I 2024-05-04 22:03:54,567] A new study created in memory with name: no-name-60e98725-35d8-427a-8110-b7a3ebf45ff9
[I 2024-05-04 22:03:58,577] Trial 0 finished with value: 0.7624496505418309 and parameters: {'n_estimators': 800, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7624496505418309.


Best trial:
  Value: 0.762
  Params:  {'n_estimators': 800, 'max_depth': 18, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}


In [43]:
#optuna ilə optimizasiya olunmuş modelləri bir listə yığıb çağırıram

models_opt=[]


models_opt.extend([
    ('XGB_optuna', best_xgb_model),
    ('LGBM_optuna', best_lgb_model),
    ('CatBoost_optuna', best_cb_model),
    ('RandomForest_optuna', best_rf_model), 
])

In [44]:
models_opt

[('XGB_optuna',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=0.606508612126674, device=None,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=5, grow_policy=None,
                importance_type=None, interaction_constraints=None,
                learning_rate=0.038646937267731225, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=7, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=790, n_jobs=None,
                num_parallel_tree=None, random_state=None, ...)),
 ('LGBM_optuna',
  LGBMClassifier(learning_rate=0.06023149379741604, max_depth=4, n_estimators=119,
                 num_leaves=51)),
 ('CatBoost_optuna', <catboost

In [45]:
#optimizasiya olunmuş modellərin gini dəyərini bir dataframe-də görə bilərik

gini_df_opt=pd.DataFrame(columns=['model', 'gini'])

for model_name, model in models_opt:
    if model_name=='RandomForest_optuna':
        x_train_filled=x_train.copy()
        x_test_filled=x_test.copy()
        for i in x_train_filled.columns:
            if x_train_filled[i].dtypes==object:
                x_train_filled[i]=x_train_filled[i].fillna(value=x_train_filled[i].mode()[0])
                x_test_filled[i]=x_test_filled[i].fillna(value=x_test_filled[i].mode()[0])
            else:
                x_train_filled[i]=x_train_filled[i].fillna(value=x_train_filled[i].mean())
                x_test_filled[i]=x_test_filled[i].fillna(value=x_test_filled[i].mean())
        gini=train_and_evaluate_model(model_name, model, x_train_filled, y_train, x_test_filled, y_test)
    else:
        gini=train_and_evaluate_model(model_name, model, x_train, y_train, x_test, y_test)
    if gini is not None:
        gini_df_opt=pd.concat([gini_df_opt, pd.DataFrame({ 'model':[model_name], 'gini':[gini]})], ignore_index=True)
        
gini_df_opt_sorted=gini_df_opt.sort_values(by='gini', ascending=False)
gini_df_opt_sorted

Model Performance for XGB_optuna
Gini prob is 64.48217781995709
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 79, number of negative: 7008
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000168 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 815
[LightGBM] [Info] Number of data points in the train set: 7087, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011147 -> initscore=-4.485360
[LightGBM] [Info] Start training from score -4.485360
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

,model,gini
3,RandomForest_optuna,0.680184
2,CatBoost_optuna,0.651132
0,XGB_optuna,0.644822
1,LGBM_optuna,0.626876


In [46]:
#həm default modellərin, həm də optimizasiya olunmuş modellərin gini dəyərlərini bir yerdə görmək üçün hər birini bir dataframe-ə yığıram

final_review = pd.concat([gini_df_sorted,gini_df_opt_sorted], axis=0)

final_review_sorted = final_review.sort_values(by='gini', ascending=False)
final_review_sorted.reset_index(drop=True, inplace=True)

final_review_sorted

,model,gini
0,RandomForest_optuna,0.680184
1,LGBM_def,0.666145
2,RandomForest_def,0.663743
3,CatBoost_optuna,0.651132
4,XGB_optuna,0.644822
5,LGBM_optuna,0.626876
6,XGB_def,0.624063
7,CatBoost_def,0.614179
8,CatBoost_cat_def,0.575893


In [47]:
#stacking modelini qurmaq üçün import edirik
from mlxtend.classifier import StackingCVClassifier

In [52]:
#stacking modelini qurmaq üçün sonuncu dataframe-dən ən yaxşı nəticə gostərən 3 modeli seçib, ilk sırada olanı meta classifier kimi, 
#digərlərini isə classifiers kimi istifadə edirəm

clf1=best_ct_model
clf2=best_xgb_model

In [53]:
meta_classifier=lgb_def_model

In [54]:
stacking_classifier = StackingCVClassifier(classifiers=[clf1, clf2],
                                           meta_classifier=meta_classifier,
                                           cv=5,
                                           use_probas=True,
                                           use_features_in_secondary=True,
                                           verbose=1,
                                           random_state=42)

In [55]:
train_and_evaluate_model('Stacking Classifier with LGBM, CatBoost and XGB', stacking_classifier, x_train, y_train, x_test, y_test)

Fitting 2 classifiers...
Fitting classifier1: catboostclassifier (1/2)
0:	learn: 0.6670123	total: 4.02ms	remaining: 1.55s
1:	learn: 0.6424788	total: 7.86ms	remaining: 1.51s
2:	learn: 0.6178498	total: 11.5ms	remaining: 1.47s
3:	learn: 0.5932557	total: 14.5ms	remaining: 1.39s
4:	learn: 0.5724239	total: 17.3ms	remaining: 1.32s
5:	learn: 0.5506873	total: 19ms	remaining: 1.21s
6:	learn: 0.5305896	total: 21.3ms	remaining: 1.16s
7:	learn: 0.5089267	total: 24.6ms	remaining: 1.16s
8:	learn: 0.4894821	total: 27.6ms	remaining: 1.16s
9:	learn: 0.4713780	total: 30ms	remaining: 1.13s
10:	learn: 0.4542035	total: 32.9ms	remaining: 1.12s
11:	learn: 0.4363842	total: 35.8ms	remaining: 1.12s
12:	learn: 0.4210736	total: 38ms	remaining: 1.09s
13:	learn: 0.4065259	total: 41.2ms	remaining: 1.1s
14:	learn: 0.3915482	total: 43.5ms	remaining: 1.08s
15:	learn: 0.3782382	total: 46.5ms	remaining: 1.08s
16:	learn: 0.3653349	total: 49ms	remaining: 1.07s
17:	learn: 0.3514769	total: 52ms	remaining: 1.06s
18:	learn: 0.3

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


68:	learn: 0.0907549	total: 184ms	remaining: 846ms
69:	learn: 0.0894351	total: 187ms	remaining: 845ms
70:	learn: 0.0881256	total: 190ms	remaining: 844ms
71:	learn: 0.0865658	total: 193ms	remaining: 843ms
72:	learn: 0.0850131	total: 196ms	remaining: 842ms
73:	learn: 0.0839075	total: 198ms	remaining: 836ms
74:	learn: 0.0823997	total: 201ms	remaining: 835ms
75:	learn: 0.0811683	total: 204ms	remaining: 834ms
76:	learn: 0.0799642	total: 208ms	remaining: 836ms
77:	learn: 0.0786488	total: 211ms	remaining: 835ms
78:	learn: 0.0773693	total: 215ms	remaining: 838ms
79:	learn: 0.0762241	total: 218ms	remaining: 837ms
80:	learn: 0.0752699	total: 221ms	remaining: 835ms
81:	learn: 0.0743878	total: 224ms	remaining: 834ms
82:	learn: 0.0735291	total: 227ms	remaining: 831ms
83:	learn: 0.0727012	total: 230ms	remaining: 829ms
84:	learn: 0.0718674	total: 233ms	remaining: 826ms
85:	learn: 0.0709650	total: 236ms	remaining: 825ms
86:	learn: 0.0702594	total: 238ms	remaining: 821ms
87:	learn: 0.0696041	total: 239

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier2: xgbclassifier (2/2)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.1s finished


0:	learn: 0.6666696	total: 3.96ms	remaining: 1.53s
1:	learn: 0.6413335	total: 7.76ms	remaining: 1.49s
2:	learn: 0.6166691	total: 11.3ms	remaining: 1.45s
3:	learn: 0.5918694	total: 14.8ms	remaining: 1.41s
4:	learn: 0.5708554	total: 18.4ms	remaining: 1.41s
5:	learn: 0.5493910	total: 21ms	remaining: 1.33s
6:	learn: 0.5292590	total: 24.1ms	remaining: 1.31s
7:	learn: 0.5075818	total: 27.9ms	remaining: 1.32s
8:	learn: 0.4882211	total: 31.2ms	remaining: 1.31s
9:	learn: 0.4700581	total: 33.8ms	remaining: 1.27s
10:	learn: 0.4527498	total: 37.5ms	remaining: 1.28s
11:	learn: 0.4350294	total: 41.2ms	remaining: 1.29s
12:	learn: 0.4197009	total: 44.3ms	remaining: 1.27s
13:	learn: 0.4048351	total: 48.1ms	remaining: 1.28s
14:	learn: 0.3898499	total: 51.1ms	remaining: 1.27s
15:	learn: 0.3766034	total: 54.9ms	remaining: 1.27s
16:	learn: 0.3637825	total: 57.6ms	remaining: 1.25s
17:	learn: 0.3500609	total: 61.3ms	remaining: 1.26s
18:	learn: 0.3375768	total: 64.9ms	remaining: 1.26s
19:	learn: 0.3255930	tot

0.6976599037285931

In [59]:
#stackin classifier etdikdən sonra ən yaxşı nəticə göstərmiş modeli univariate analysis edirəm. əgər overfitting etmiş dəyişənlər varsa onlari modeldən çıxarıb yenidən model qururam

variables = []
train_gini_scores = []
test_gini_scores = []


for i in x_train.columns:
    x_train_single_var = x_train[[i]]
    x_test_single_var = x_test[[i]]

    best_lgb_model.fit(x_train_single_var, y_train)
    y_pred_train_single_var = best_lgb_model.predict_proba(x_train_single_var)[:, 1]
    train_roc = roc_auc_score(y_train, y_pred_train_single_var)
    train_gini = 2 * train_roc - 1

    y_pred_test_single_var = best_lgb_model.predict_proba(x_test_single_var)[:, 1]
    test_roc = roc_auc_score(y_test, y_pred_test_single_var)
    test_gini = 2 * test_roc - 1

    variables.append(i)
    train_gini_scores.append(train_gini)
    test_gini_scores.append(test_gini)

results_df = pd.DataFrame({
    'Variable': variables,
    'Train Gini': train_gini_scores,
    'Test Gini': test_gini_scores
})

results_df_sorted = results_df.sort_values(by='Test Gini', ascending=False)

pd.options.display.float_format = '{:.4f}'.format

results_df_sorted

[LightGBM] [Info] Number of positive: 79, number of negative: 7008
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 7087, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011147 -> initscore=-4.485360
[LightGBM] [Info] Start training from score -4.485360
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

,Variable,Train Gini,Test Gini
1,oldbalanceOrg,0.7699,0.4593
3,unusuallogin,0.4748,0.4534
0,amount,0.7797,0.4443
10,type_PAYMENT,0.3575,0.3034
2,newbalanceOrig,0.6332,0.2643
11,type_TRANSFER,0.2241,0.2203
8,type_CASH_OUT,0.2394,0.2003
7,transaction_day,0.2070,0.1668
13,Time of day_Morning,0.0417,0.0530
9,type_DEBIT,0.0217,0.0347


In [60]:
#overfitting etmiş etmiş müstəqil dəyişənləri nəzərə almayıb digərləri ilə yenidən model qururam

fin_input=data_dummy[['oldbalanceOrg', 'amount', 'unusuallogin', 'type_PAYMENT', 'newbalanceOrig', 'type_TRANSFER', 'type_CASH_OUT', 'transaction_day']]
fin_output=data_dummy['isFraud']

In [61]:
data_dummy.columns

Index(['amount', 'oldbalanceOrg', 'newbalanceOrig', 'unusuallogin',
       'isFlaggedFraud', 'isFraud', 'transaction_year', 'transaction_month',
       'transaction_day', 'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT',
       'type_TRANSFER', 'Acct type_Savings', 'Time of day_Morning',
       'Time of day_Night'],
      dtype='object')

In [62]:
x_train_fin, x_test_fin, y_train_fin, y_test_fin=train_test_split(fin_input, fin_output, test_size=0.3, random_state=42)

In [63]:
lgb_fin=best_lgb_model.fit(x_train_fin, y_train_fin)

[LightGBM] [Info] Number of positive: 79, number of negative: 7008
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 7087, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011147 -> initscore=-4.485360
[LightGBM] [Info] Start training from score -4.485360
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [64]:
train_and_evaluate_model('LGB with univariate', lgb_fin, x_train_fin, y_train_fin, x_test_fin, y_test_fin)

[LightGBM] [Info] Number of positive: 79, number of negative: 7008
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000189 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 7087, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.011147 -> initscore=-4.485360
[LightGBM] [Info] Start training from score -4.485360
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

0.659347987790802